# Cross-species analysis: cyno and human lung
Dan Carvalho

### Settings

In [1]:
import sys
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import glob
import re
import scvi
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.sparse import csr_matrix
from scipy import stats
import leidenalg

/home/carvadan/.local/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/carvadan/.local/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


#### Homology data frame
The first step is to create a homology data frame. This will be used, later on, to merge and filter the AnnData objects of cyno and human.

In [2]:
# read in the homology tsv table from BioMart (website, not R package)
homology = pd.read_table('/home/carvadan/mart_export.tsv')
# filter by one-to-one, one-to-many, many-to-many
o2o = homology[homology['Crab-eating macaque homology type'] == 'ortholog_one2one']
o2m = homology[homology['Crab-eating macaque homology type'] == 'ortholog_one2many']
m2m = homology[homology['Crab-eating macaque homology type'] == 'ortholog_many2many']
# merge all into one by row binding
homology_df = pd.concat([o2o, o2m, m2m], axis = 0)

In [5]:
len(gene_mapping3)

19667

#### Create a gene mapper

The ```zip()``` function here is neat. It takes at least 2 lists and turns them into a list of tuples. We then pass that inside the ```dict()``` function and it automatically creates a dictionary for us. No need to mess around with for-loops so assign key-value pairs from each list. Great function!

**These are some gene mappings I think I will need for the analysis.**

In [4]:
# gene mapper
gene_mapping = dict(zip(homology_df['Gene stable ID'], homology_df['Crab-eating macaque gene name']))
gene_mapping2 = dict(zip(homology_df['Gene stable ID'], homology_df['Gene name']))
gene_mapping3 = dict(zip(homology_df['Gene name'], homology_df['Crab-eating macaque gene name']))
gene_mapper = dict(zip(homology_df['Crab-eating macaque gene name'], homology_df['Gene name']))

#### Mitochondrial Genes

These are the mitochondrial gene names for cynomolgus monkey. I pull this from NCBI; however, it would probably be easier to do on Ensembl's BioMart online.

In [ ]:
# mitochondrial genes for cynomolgus monkey
mito_genes = ["KEG98_t01", "KEG98_r02", "KEG98_t02", "KEG98_r01", 
              "KEG98_t03", "ND1", "KEG98_t04", "KEG98_t05", 
              "KEG98_t06", "ND2", "KEG98_t07", "KEG98_t08", 
              "KEG98_t09", "KEG98_t10", "KEG98_t11", "COX1", 
              "KEG98_t12", "KEG98_t13", "COX2", "KEG98_t14", 
              "ATP8", "ATP6", "COX3", "KEG98_t15", "ND3", 
              "KEG98_t16", "ND4L", "ND4", "KEG98_t17", 
              "KEG98_t18", "KEG98_t19", "ND5", "ND6", 
              "KEG98_t20", "CYTB", "KEG98_t21", "KEG98_t22"]

These are the Ensembl gene IDs for all the human mitochondrial genes (I pulled these from BioMart [online]; 37 total).

In [ ]:
# ENSEMBL mito genes
ENS_mt = ['ENSG00000210049', 'ENSG00000211459', 'ENSG00000210077', 'ENSG00000210082', 
          'ENSG00000209082', 'ENSG00000198888', 'ENSG00000210100', 'ENSG00000210107', 
          'ENSG00000210112', 'ENSG00000198763', 'ENSG00000210117', 'ENSG00000210127', 
          'ENSG00000210135', 'ENSG00000210140', 'ENSG00000210144', 'ENSG00000198804', 
          'ENSG00000210151', 'ENSG00000210154', 'ENSG00000198712', 'ENSG00000210156', 
          'ENSG00000228253', 'ENSG00000198899', 'ENSG00000198938', 'ENSG00000210164', 
          'ENSG00000198840', 'ENSG00000210174', 'ENSG00000212907', 'ENSG00000198886', 
          'ENSG00000210176', 'ENSG00000210184', 'ENSG00000210191', 'ENSG00000198786', 
          'ENSG00000198695', 'ENSG00000210194', 'ENSG00000198727', 'ENSG00000210195', 
          'ENSG00000210196']

## Cyno data

First, let's set the path to the cyno data

In [ ]:
# path to matrices
FC_path = '/home/carvadan/female_cyno/filtered_feature_bc_matrix/'
MC_path = '/home/carvadan/male_cyno/filtered_feature_bc_matrix/'

Now we can create the AnnData objects for each matrix by using Scanpy
`sc.read_h5ad(...)`

In [ ]:
# create AnnData
FC = sc.read_10x_mtx(FC_path)
MC = sc.read_10x_mtx(MC_path)

Let's check out each AnnData object

In [ ]:
FC

In [ ]:
MC

Contrary to Seurat in R, AnnData objects (what Scanpy uses) are set up differently. In Seurat, we have the genes as rows and the cells as columns; in Scanpy (via AnnData) we have the rows as cell and the columns as genes. This is something to keep in mind! We may have to transpose these count matrices which is easily performed by appending `.T` to the end of the count matrix (which is `nameofobject.X`)

### Doublet removal

This is optional, but preferred. I would like for this analysis to be of high-quality; let's do it.

This method will be using `scvi-tools`. We will train a machine learning model to do the doublet detection. The thing to note about this is that the data cannot be concatenated yet; we cannot merge the two cyno datasets OR the human data. This is mainly due to batch effects that could affect the ability to distinguish a doublet from a singlet. Therefore, I will be doing the analysis on the two cyno (male and female) datasets separately before merging them. For the human I will merge them then do this step. This is because they come from the same study, so we shouldn't have to worry aobut batch effects too much.

#### Filter down the genes

In [ ]:
# here we are only keeping genes that are found in AT LEAST 10 cells

sc.pp.filter_genes(FC, min_cells = 10)
sc.pp.filter_genes(MC, min_cells =10)

In [ ]:
# let's look at both objects again
print(FC, '\n\n', MC)

As we can see, it removed a lot of the genes that were not expressed in at least 10 cells.

#### Keep top 2,000 variable genes

In [ ]:
# let's go ahead and keep the top 2,000 vaiable genes that more-or-less describe the data the best for both female and male cyno
sc.pp.highly_variable_genes(FC, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
sc.pp.highly_variable_genes(MC, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')

So, if we look at the AnnData objects again we will see that we are down to 2,000 cells. Let's have a look below.

In [ ]:
print(FC, '\n\n', MC)

### Train `scvi` model for doublet prediction

Great! Now we want to set up a `scvi` model so that we can then enable doublet prediction. It will be a very simple model set-up.

#### For female

In [ ]:
scvi.model.SCVI.setup_anndata(FC)
vae_1 = scvi.model.SCVI(FC)
vae_1.train()

Now we can go ahead and train the `SOLO` model which detects doublets

In [ ]:
solo_1 = scvi.external.SOLO.from_scvi_model(vae_1)
solo_1.train()

Let's look at the predictions

In [ ]:
solo_1.predict()

We will make a new data frame from the above data frame, and add a new column to label what the predictions are actually saying

In [ ]:
# call the new data frame df1
df1 = solo_1.predict()
# create new column
df1['prediction'] = solo_1.predict(soft = False)

# may need to uncomment code under here in case scvi adds `-0` to the end of the barcodes
# df1.index = df1.index.map(lambda x: x[:-2])
df1

Let's look at the data now

In [ ]:
df1.groupby('prediction').count()

Now we can see how many were labelled as doubles and singlets. We need to be sure before we just throw away cells that may actually be singlets.

In [ ]:
# add a new difference column
df1['dif'] = df1.doublet - df1.singlet
df1

Let's plot the distribution to see what the data looks like

We will use `seaborn` (imported as `sns`) to visualize only the doublets, where x is the difference between doublets and singlets

In [ ]:
sns.displot(df1[df1.prediction == 'doublet'], x = 'dif')

Creating a new dataframe named `doublets` and filtering out cells above 1

In [ ]:
doublets1 = df1[(df1.prediction == 'doublet') & (df1.dif > 1)]
doublets1

This dataframe will contain the corresponding cell barcodes and the doublets. Therefore, we can use this dataframe (since it has cell barcodes) to directly filter the fresh, reloaded AnnData object which will be shown in the cell below.

In [ ]:
FC = sc.read_10x_mtx(FC_path)
FC

Now we have the fresh AnnData object. Let's remove the doublets!

In [ ]:
FC.obs['doublet'] = FC.obs.index.isin(doublets1.index)

Showing that the `doublet` column was successfully created.

In [ ]:
FC.obs

Now we can filter for the ones that are false! Here, we use the tilde (~) to keep only the singlets which are denoted as 'False' in the `doublets` column.

In [ ]:
FC = FC[~FC.obs.doublet] # the `~` means you keep the ones that are False

Add a sample identifier for the integration step later in this analysis.

In [ ]:
# add identifier
FC.obs['Sample'] = 'female'

Making sure it worked...

In [ ]:
FC.obs

Now we have our fresh, raw, AnnData object with all the doublets removed!

Let's save this for use later.

In [ ]:
FC.write_h5ad(filename = 'raw_female_cyno.h5ad')

#### For male

**Note: The workflow here is _exactly_ the same as what we did for the female cyno above. I am leaving out the comments on what's going on for the sake of brevity. If a refresher of what's happening is needed, just take a look above. :)**

In [ ]:
scvi.model.SCVI.setup_anndata(MC)
vae_2 = scvi.model.SCVI(MC)
vae_2.train()

In [ ]:
solo_2 = scvi.external.SOLO.from_scvi_model(vae_2)
solo_2.train()

In [ ]:
solo_2.predict()

In [ ]:
# call the new data frame df1
df2 = solo_2.predict()
# create new column
df2['prediction'] = solo_2.predict(soft = False)

# may need to uncomment code under here in case scvi adds `-0` to the end of the barcodes
# df1.index = df1.index.map(lambda x: x[:-2])
df2

In [ ]:
df2.groupby('prediction').count()

In [ ]:
# add a new difference column
df2['dif'] = df2.doublet - df2.singlet
df2

In [ ]:
sns.displot(df1[df1.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets2 = df2[(df2.prediction == 'doublet') & (df2.dif > 1)]
doublets2

In [ ]:
MC

In [ ]:
MC = sc.read_10x_mtx(MC_path)
MC

In [ ]:
MC.obs['doublet'] = MC.obs.index.isin(doublets2.index)

In [ ]:
MC.obs

In [ ]:
MC = MC[~MC.obs.doublet] # the `~` means you keep the ones that are False

In [ ]:
MC

In [ ]:
# add identifier
MC.obs['Sample'] = 'male'

In [ ]:
MC.write_h5ad(filename = 'raw_male_cyno.h5ad')

### Preprocessing

#### Female

In [ ]:
FC.var

First thing we need to do is prepend 'MT-' to all the mitochondrial genes

In [ ]:
# iterate over the index and modify the gene names
for gene in FC.var.index:
    if gene in mito_genes:
        FC.var_names = FC.var_names.to_series().replace(gene, 'MT-' + gene)

In [ ]:
FC.var['gene_ids'] = FC.var.index
FC.var

Now we can begin the preprocessing

In [ ]:
# labeling True or False if they start with MT
FC.var['mt'] = FC.var.index.str.startswith('MT-')
FC.var

In [ ]:
# calculate some QC metrix here
sc.pp.calculate_qc_metrics(FC, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)

FC.var

In [ ]:
FC.obs

In [ ]:
# sort by number of cells where any gene was found
FC.var.sort_values('n_cells_by_counts')

In [ ]:
# filter out genes that were not in at least 3 cells
sc.pp.filter_genes(FC, min_cells = 3)

In [ ]:
FC.var.sort_values('n_cells_by_counts')

Let's look at total counts

In [ ]:
FC.obs.sort_values('total_counts')

Lowest counts are 500 which is good. No need to filter that.

In [ ]:
# sc.pp.filter_cells(FC, min_genes = ...)

Now let's check out the genes per cell

In [ ]:
FC.obs.sort_values('n_genes_by_counts')

Need to filter this to 200

In [ ]:
sc.pp.filter_cells(FC, min_genes = 200)

In [ ]:
# just checking
FC.obs.sort_values('n_genes_by_counts')

Let's go ahead and plot these QC metrics.

In [ ]:
sc.pl.violin(FC, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
            jitter = 0.4, multi_panel = True)

Based on these QC plots, we can filter our data and remove dying cells, outliers, etc.

We can be more objective in filtering by using numpy. We can filter with quantiles rather than picking arbirarily.

In [ ]:
upper_lim = np.quantile(FC.obs.n_genes_by_counts, 0.98)

# if you wanted to pick, we could do this: upper_lim = 3000
upper_lim

In [ ]:
# filtering out all cells below `upper_lim`

FC = FC[FC.obs.n_genes_by_counts < upper_lim]
FC.obs

In [ ]:
# filter mito
FC = FC[FC.obs.pct_counts_mt < 10]
FC.obs

#### Male

In [ ]:
MC.var

In [ ]:
# iterate over the index and modify the gene names
for gene in MC.var.index:
    if gene in mito_genes:
        MC.var_names = MC.var_names.to_series().replace(gene, 'MT-' + gene)

In [ ]:
# reseeting gene_ids column to index values

MC.var['gene_ids'] = MC.var.index
MC.var

In [ ]:
# labeling True or False if they start with MT
MC.var['mt'] = MC.var.index.str.startswith('MT-')
MC.var

In [ ]:
# calculate some QC metrix here
sc.pp.calculate_qc_metrics(MC, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)

MC.var

In [ ]:
FC.obs

In [ ]:
# sort by number of cells where any gene was found
MC.var.sort_values('n_cells_by_counts')

In [ ]:
# filter out genes that were not in at least 3 cells
sc.pp.filter_genes(MC, min_cells = 3)

In [ ]:
MC.var.sort_values('n_cells_by_counts')

In [ ]:
MC.obs.sort_values('total_counts')

Again, not needed as lowest is 500

In [ ]:
sc.pp.filter_cells(MC, min_genes = 200)

In [ ]:
# just checking
MC.obs.sort_values('n_genes_by_counts')

In [ ]:
sc.pl.violin(MC, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
            jitter = 0.4, multi_panel = True)

In [ ]:
upper_lim = np.quantile(MC.obs.n_genes_by_counts, 0.98)

# if you wanted to pick, we could do this: upper_lim = 3000\
upper_lim

In [ ]:
# filtering out all cells below `upper_lim`
MC = MC[MC.obs.n_genes_by_counts < upper_lim]
MC.obs

In [ ]:
# filter mito
MC = MC[MC.obs.pct_counts_mt < 10]
MC.obs

### Normalization

**Very important step!** In scRNA-seq there a lot of variation exists between cells, even between cells of the same type (mainly due to sequencing biases, etc). Normalization allows us to compare genes. We will normalize the counts in each cell so that their total counts add up to the same value.

We can quickly show why we need to do normalization if we sum the counts for each respective cell

In [ ]:
FC.X.sum(axis = 1)

#### Female

In [ ]:
sc.pp.normalize_total(FC, target_sum = 1e4) # normalize every cell to 10,000 UMI

In [ ]:
FC.X.sum(axis = 1)

Now we see the sum for each cell all adds up to 10,000!

Changing them to log counts

In [ ]:
sc.pp.log1p(FC) # change to log counts

In [ ]:
FC.X.sum(axis = 1)

After log-transformation they are not the same number; however this is because log-transforming something is not a linear operation. Anyway, the numbers are much more comparable now compared to the non-normalized data.

**It is very important to save (freeze) the data as it is now before we do any further processing.**

Adding identifier

In [ ]:
FC.obs['Sample'] = 'female'

In [ ]:
FC.raw = FC # save the raw data

This saves it into the `raw` slot in the AnnData object

#### Male

In [ ]:
sc.pp.normalize_total(MC, target_sum = 1e4) # normalize every cell to 10,000 UMI

In [ ]:
MC.X.sum(axis = 1)

In [ ]:
sc.pp.log1p(MC)

MC.X.sum(axis = 1)

Adding identifier

In [ ]:
MC.obs['Sample'] = 'male'

In [ ]:
MC.raw = MC

**Saving these AnnData objects**

In [ ]:
FC.write_h5ad(filename = 'female_cyno.h5ad')
MC.write_h5ad(filename = 'male_cyno.h5ad')

## Human data

We will essentially repeat the process we did above for the cyno data but for the human data, with some differences, of course.

I will make a preprocessing function up here that is slightly different than the one we have below. This one combines the doublet detection and then will run through the same preprocessing analysis as above.

#### Preprocessing Functions

In [ ]:
def doublet(file):
    adata = sc.read_h5ad(file)
    sc.pp.filter_genes(adata, min_cells = 10)
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train(batch_size = 129) # first sample is 128, so it will be '1' in the model. Just changed this so it'd run
    df = solo.predict()
    df['prediction'] = solo.predict(soft = False)
    # df.index = df.index.map(lambda x: x[:-2])
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
    # re-read in the data and then filter out doublets
    adata = sc.read_h5ad(file)
    adata.obs['Sample'] = file.split('_')[9] + '_' + file.split('_')[5]
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    # remove all doublets
    # remember that the `~` mean select all that have the Boolean value of False
    adata = adata[~adata.obs.doublet]
    
    return adata

In [ ]:
def pp(adata):

    sc.pp.filter_cells(adata, min_genes=200) #get rid of cells with fewer than 200 genes
    #sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
    adata.var['mt'] = adata.var_names.isin(mito_genes)  # annotate the group of mitochondrial genes as 'mt'
    
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 10]

    return adata

In [ ]:
def pp_for_doublet(adata):

    # now the classic preprocessing
    sc.pp.filter_cells(adata, min_genes = 200) # gets rid of cells with fewer than 200 genes
    #sc.pp.filter_genes(adata, min_cells = 3) # usually have this here, but will be doing it later
    adata.var['mt'] = adata.var_names.isin(ENS_mt) # annotate mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(adata, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)
    # use numpy for quantiles
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, 0.98)
    adata = adata[adata.obs.n_genes_by_counts.values < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 10]
    
    return adata

Let's get all the `.h5ad` files that we need

#### Use Regular Expressions To Get Human Files

In [ ]:
# path to directory
directory = '/home/carvadan/human_lung_analysis/human_normal_lung_GSE136831_scRNA-seq_h5ad_result/'

# regular expression patter
pattern = r'.*\.h5ad$'

# use glob with regex pattern
# use of the glob module to search for files in the specified diretory path
# the `'*'` wildcard is used to match any file name in the directory
# file for file in glob.glob(directory + '*') is a list comprehension that iterates over the files obtained from glob.glob(directory + '*')
# and assigns each file path to the variable `file` creating a new list containing the file paths

# `if re.match(pattern, file)` is a conditional. So only those files that match the pattern are added to the list mentioned above.
file_list = [file for file in glob.glob(directory + '*') if re.match(pattern, file)]

# for file in file_list:
#     print(file)

In [ ]:
file_list

In [ ]:
# gene_symbols = []

# for i in adata.var_names:
#     if i in gene_mapping4:
#         gene_symbols.append(gene_mapping4[i])
#     else:
#         gene_symbols.append(i)


# adata.var['gene_symbols'] = gene_symbols
# adata.var['ensemble'] = adata.var_names
# adata.var_names = adata.var['gene_symbols']
    
# adata.var_names

Now let's try running for all files.

In [ ]:
out = []
for i, file in enumerate(file_list):
    print(i)
    out.append(doublet(file))

Now that everything has run and is already a list, let's make sure that it worked correctly.

In [ ]:
out

In [ ]:
for i, adata in enumerate(out):
    print(i)
    pp_for_doublet(adata)
    print('\n')

Since they're all AnnData objects, we can now concatenate them into one object.

In [ ]:
human = ad.concat(out)
human.obs_names_make_unique

Save the data as `.h5ad`

In [ ]:
human.write_h5ad(filename = 'human_combined.h5ad')

## Concatenate human and cyno together

Since we now have the concatenated cynomolgus monkey and human AnnData objects, we need to filter them. This will be performed by making a column in the cyno object called `gene_symbols` followed by `merged = cyno.concatenate(human)`. This will automatically subset the object of the union of genes in both objects via an inner join. Then we should be free to integrate once this is completed.

Preprocess the cyno data since we onlt did doublet detection and removal

In [ ]:
cyno = sc.read_h5ad('cyno_combined.h5ad')
cyno.obs_names_make_unique

In [ ]:
human = sc.read_h5ad('human_combined.h5ad')
human.obs_names_make_unique

### Filter for Orthologoues Only

#### Create `gene_symbol` column for cyno

In [ ]:
homology_indicator_c = cyno.var_names.isin(gene_mapping.values())

In [ ]:
homology_indicator_c

In [ ]:
cyno = cyno[:, homology_indicator_c]

In [ ]:
cyno.var

Subset to only those in `homology_indicator_c`

In [ ]:
# add hgnc, index there
hgnc = []
for i in cyno.var_names:
    if i in gene_mapper:
        hgnc.append(gene_mapper[i])
cyno.var['gene'] = hgnc

In [ ]:
cyno.var

In [ ]:
cyno.var['loc'] = cyno.var_names
cyno.var_names = cyno.var['gene']

In [ ]:
cyno.var

In [ ]:
cyno.write(filename = 'cyno_combined.h5ad')

#### Create `gene_symbol` column for human

In [ ]:
human

In [ ]:
homology_indicator_h = human.var_names.isin(gene_mapping.keys())

In [ ]:
len(homology_indicator_h)

Subset to only those in `homology_indicator_h`

In [ ]:
human = human[:, homology_indicator_h]

In [ ]:
human.var

#### Indexing human to HGNC

In [ ]:
HGNC = []
for i in human.var_names:
    if i in gene_mapping2:
        HGNC.append(gene_mapping2[i])
human.var['gene'] = HGNC

In [ ]:
human.var['ensembl'] = human.var_names
human.var_names = human.var['gene']

#### Making values unique

In [ ]:
len(new_ind)

In [ ]:
ints = [*range(61516)]
# range1 = [str(x) for x in ints]

th_obs_index = list(th.obs_names)

dictionary = dict(zip(ints, th_obs_index))


new_ind = []
for i in dictionary:
    new_ind.append(dictionary[i] + '-' + str(i))
# new_ind

In [ ]:
th.obs['index'] = new_ind
th.obs_names = th.obs['index']

In [ ]:
len(th.var_names)

In [ ]:
ints = [*range(16732)]
# range1 = [str(x) for x in ints]

th_obs_index = list(th.var_names)

dictionary = dict(zip(ints, th_obs_index))


new_ind = []
for i in dictionary:
    new_ind.append(str(dictionary[i]) + '-' + str(i))

In [ ]:
th.var['index'] = new_ind
th.var_names = th.var['index']

In [ ]:
th

In [ ]:
# gene = []
# for i in th.var_names:
#     if i in gene_mapping2:
#         gene.append(gene_mapping[i])
th.var['gene'] = gene
th.var['ensembl'] = th.var_names
th.var_names = th.var['gene']

In [ ]:
print(len(gene), len(th.var_names))

#### Concatenating

In [ ]:
human = sc.read_h5ad(filename = 'combined_human.h5ad')
cyno = sc.read_h5ad(filename = 'cyno_combined.h5ad')

In [ ]:
human.var_names_make_unique
cyno.var_names_make_unique

In [ ]:
cyno

In [ ]:
merged = sc.concat([human, cyno], axis = 1, join = 'outer', label = ['human', 'cyno'])

In [ ]:
merged

In [ ]:
merged.var

Save the merged object

In [ ]:
merged.write(filename = 'human_cyno_merged.h5ad')

## Cleaning

We finally managed to concatenate both objects into one. The issue is we had to use an outer join. Therefore, the resulting genes are more than expected. What we need to do is clean up the genes and hopefully re-index.

#### Assign Sample identity

In [ ]:
h_obs = list(human.obs.index)
c_obs = list(cyno.obs.index)

In [ ]:
merged.obs['human'] = merged.obs.index.isin(h_obs)

In [ ]:
Sample = list(merged.obs['human'])

# change to string
Sample = list(map(str, Sample))

dictionary = {'True': 'human', 'False': 'cyno'}

sample = []
for i, file in enumerate(Sample):
    if Sample[i] in dictionary:
        sample.append(dictionary[file])

sample

In [ ]:
merged.obs['Sample'] = sample

In [ ]:
del merged.obs['human']

In [ ]:
merged.obs

#### Remove nans

In [ ]:
merged.var

In [ ]:
NAs = []

for i in merged.var.index:
    if i.startswith('nan-'):
        NAs.append(i)
NAs

In [ ]:
merged.var['NAs'] = merged.var.index.isin(NAs)

In [ ]:
merged = merged[:, merged.var['NAs']]

In [ ]:
merged.var

#### Filter Homology (again)

Reset the index

In [ ]:
merged.var_names = merged.var['gene']

Check for any duplicates

In [ ]:
np.any(merged.var.duplicated())

Let's try and filter now

Taking a look at `gene_mapping3` from the beginning of the notebook

In [ ]:
len(gene_mapping3)

We want to get the gene names and create a list which we will then use to filter the `merged` object on

In [ ]:
Gene = list(merged.var['index'])

In [ ]:
gg = []
for i in Gene:
    if type(i) != float:
        gg.append(i)

In [ ]:
len(gg)

In [ ]:
merged.var['orth_gene'] = merged.var['index'].isin(gg)

Now filter the entire object where they are `True`

In [ ]:
all(merged.var['orth_gene'])

In [ ]:
merged = merged[:, merged.var['orth_gene']]
merged

In [ ]:
merged.var

In [ ]:
merged.write(filename = 'human_cyno_merged.h5ad')

## Integration

**For integration, we actually need the raw counts. We can create a preprocessing function which we will call `pp` or `pp_for_doublet`. `pp` will include all the steps that we did above and we will apply it to the combined AnnData.**

In [ ]:
# def pp(adata):
    
#     # iterate over the index and modify the gene names
#     for gene in adata.var.index:
#         if gene in mito_genes:
#             adata.var_names = adata.var_names.to_series().replace(gene, 'MT-' + gene)
        
#     # change names of `gene_ids` column
#     adata.var['gene_ids'] = adata.var.index
    
#     # labeling True or False if they start with MT
#     adata.var['mt'] = adata.var.index.str.startswith('MT-')
            
#     # now start the preprocessing
#     sc.pp.calculate_qc_metrics(adata, qc_vars = ['mt'], percent_top = None, log1p = False, inplace = True)
#     # filter out genes that were not in at least 3 cells
#     #sc.pp.filter_genes(adata, min_cells = 3)
#     sc.pp.filter_cells(adata, min_genes = 200)
    
#     # filtering out all cells below `upper_lim`
#     upper_lim = np.quantile(adata.obs.n_genes_by_counts, 0.98)
#     adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    
#     # filter mito
#     adata = adata[adata.obs.pct_counts_mt < 10]
    
#     adata = cyno
    
#     # return preprocessed concatenated AnnData
#     return cyno

Let's run for `merged`

In [ ]:
human = sc.read_h5ad('combined_human.h5ad')
cyno = sc.read_h5ad('cyno_combined.h5ad')

In [ ]:
human

In [ ]:
cyno

In [ ]:
merged = sc.read_h5ad('human_cyno_merged.h5ad')

#### Retrieve `.obs` from non-concatenated objects

Making sure all barcodes have '-1' at the end

In [ ]:
human.obs['cells'] = human.obs_names.str.split('-').str[0]
human.obs['cells'] = human.obs['cells'] + '-1'
human.obs_names = human.obs['cells']
del human.obs['cells']

In [ ]:
merged.obs['cells'] = merged.obs_names.str.split('-').str[0]

In [ ]:
merged.obs

In [ ]:
merged.obs['cells'] = merged.obs['cells'] + '-1'

In [ ]:
merged.obs_names = merged.obs['cells']
del merged.obs['cells']

In [ ]:
merged.obs

Get `.obs`

The `.obs` did not transfer during the concatenation (for whatever reason...will look into it). So what we can do is row-bind the `.obs` of human and cyno together. Then, set the obs of the merged object equal to `merged_obs_df.reindex(merged.obs.index)`. This will preserve the cell ID order in `merged`. Additionally, we can validate this by checking if all the values line up like so: `equal_elements = [x == y for x, y in zip(list(t.obs.index), list(merged.obs.index))]` and then running `any(equal_elements)`. If the result of `any(equal_elements)` is true, then we know that every element at each index of the two lists is equal to each other, and the order is preserved.

In [ ]:
mapping_human

Using `t` as a test

In [ ]:
t = merged

In [ ]:
merged_obs_df = pd.concat([human.obs, cyno.obs], axis = 0)

t.obs = merged_obs_df.reindex(t.obs.index)

In [ ]:
t.var_names = list(t.var_names)

In [ ]:
t.var_names_make_unique()

In [ ]:
equal_elements  = [x == y for x, y in zip(list(t.obs.index), list(merged.obs.index))]
all(equal_elements)

It seems to have worked. Let's do it for real with `merged`.

In [ ]:
merged.obs = merged_obs_df.reindex(merged.obs.index)

Save the object

In [ ]:
merged.write(filename = 'human_cyno_merged.h5ad')

We need to get rid of the genes that are not in any cells

In [ ]:
merged.obs_names_make_unique()

In [ ]:
sc.pp.filter_genes(merged, min_cells = 10)

In [ ]:
merged

In [ ]:
merged.X

Our matrix is already sparse; however, we can convert it to sparse (if it were dense) with the code below :

`adata.X = csr_matrix(adata.X)`

#### Save the concatenated object

In [ ]:
merged.write_h5ad(filename = 'human_cyno_merged.h5ad')

Let's look at how many cells we have for each sample

In [ ]:
merged.obs.groupby('Sample').count()

In [ ]:
merged

For integration with `scvi`, we need the number of genes to be about half the number of cells.

Our number of genes here is good. In the scenario where we would need to remove more genes, we could do so by applying a more stringent threshold like the one below.

In [ ]:
# sc.pp.filter_genes(cyno, min_cells = 100)
# cyno

Great! Now we have about half the as many genes as cells.

Now the data are combined, but we aren't done yet. We will actually perform the integration where we can correct for batch effects and mitochondrial counts.

#### Now we get to use `scvi` to do the integration

In order to prep our AnnData object (`cyno`) for use with `scvi` we need to save the raw data as it is now

In [ ]:
merged = sc.read_h5ad('human_cyno_merged.h5ad')

In [ ]:
sc.pp.filter_cells(merged, min_counts = 1)

In [ ]:
# saving raw data
merged.layers['counts'] = merged.X.copy()

**This data has not been normalized, converted to log counts, or anything. Just saving the raw data into this layer, `counts`. We won't touch it again *EXCEPT* when we read from it.**

Making a quick normalization function

#### Normalization

In [ ]:
sc.pp.normalize_total(merged, target_sum = 1e4)
sc.pp.log1p(merged)

Now save this log-normalized data into the `raw` slow

In [ ]:
merged.raw = merged

`SCVI` will use the raw counts we saved above, and other modules and functions will use the log-normalized counts

In [ ]:
merged.obs.groupby('Sample').count()

Let's add a `species` column to `.obs` to denote cyno and human

In [ ]:
# initialize empty list
species = []
# set regex pattern
pattern = r'^\d{3}_GSE\d{6}$'
for sample in merged.obs['Sample']:
    if re.match(pattern, sample):
        species.append('human')
    else:
        species.append('cyno')
merged.obs['species'] = species
merged.obs

#### Set up the `SCVI` model

In [ ]:
scvi.model.SCVI.setup_anndata(merged, layer = 'counts',
                             categorical_covariate_keys = ['Sample', 'species'],
                             continuous_covariate_keys = ['pct_counts_mt', 'total_counts'])

#### Initialize and train the model

In [ ]:
model = scvi.model.SCVI(merged)
model.train()

Now that the model is trained we want to get a couple of things from it:
    
    1. We can use `model.get_latent_representation()` which is a numpy array were the number of cells are equal to the number of rows, and there are 10 columns. This is the `SCVI` array that represents our data. This is what we will be using for clustering and UMAP

In [ ]:
model.get_latent_representation()

Let's look at the dimensions

In [ ]:
model.get_latent_representation().shape

Now, we will save it to the AnnData object

In [ ]:
merged.obsm['X_scVI'] = model.get_latent_representation()

We can also get the `SCVI` normalized expression, which is a cell by gene data frame and save it as a layer rather than overwriting the default values

In [ ]:
merged.layers['scVI_normalized'] = model.get_normalized_expression(library_size = 1e4)

Saving so I don't have to run the model again :)

In [ ]:
model.save('model/')

In [ ]:
merged.write_h5ad(filename = 'human_cyno_merged.h5ad')

### Clustering

Now we can go ahead and cluster the data

Loading in the object

In [ ]:
merged = sc.read_h5ad('human_cyno_merged.h5ad')

In [ ]:
merged

#### Using `sc.pp,neighbors()` function for clustering

We are using the latent representation from `scVI` to calculate the neighbors

In [ ]:
sc.pp.neighbors(merged, use_rep = 'X_scVI')

Since the neighbords were calculated above using `X_scVI` we don't have to specify anything while passing `sc.tl.umap(merged)` and the same for `sc.tl.leiden(merged)` except with a resolution of 0.5 (which we might change later). This might take a minute or two depending on how many cells you have!

We are using the Leiden algorithm here which is an improvement upon the largely used Louvain algorithm. The Leiden algorithm has been proposed as a great algorithm for scRNA-seq analysis in place of Louvain. Vincent Traag, Ludo Waltman, and Nees Jan van Eck created this algorithm as they noticed a serious issue with the Louvain algorithm— the Louvain algorithm may yield arbitrarily badly connected communities. In the worst case, communities may even be disconnected, especially when running the algorithm iteratively. This Leiden algorithm corrects that and guarantees connectedness of communities which is crucial for single-cell analysis.

In [ ]:
sc.tl.umap(merged)
sc.tl.leiden(merged, resolution = 0.5)

We will plot two UMAPs

In [ ]:
sc.pl.umap(merged, color = ['leiden', 'Sample', 'species'], frameon = True)

#### Save object as `integrated.h5ad`

In [ ]:
merged.write(filename = 'integrated.h5ad')

## Label Cell-Types

Switching the name of `merged` to `integrated`

In [ ]:
integrated = merged
del merged

Make sure it worked

In [ ]:
integrated

We will be using both the `scVI` trained model data.

#### `scVI` Data

Reload the model and AnnData (if needed)

In [ ]:
integrated = sc.read_h5ad('integrated.h5ad')

In [ ]:
model = scvi.model.SCVI.load('model/', adata = integrated, use_gpu = False)

In [ ]:
scVI_markers = model.differential_expression(groupby = 'leiden')

Let's look at the markers and save them to load later

In [ ]:
scVI_markers.to_csv('scVI_markers', sep = '\t', index = False)

In [ ]:
scVI_markers

The dataframe contains a column 'is_de_fdr_0.05' which we only want the genes where their value for this column is `True` and we want the log-fold change (lfc) to be greater than 0.5. Let's filter for that below.

In [ ]:
scVI_markers = scVI_markers[(scVI_markers['is_de_fdr_0.05']) & (scVI_markers.lfc_mean > .5)]
# create a names column of the index so it is easier to filter the DF
scVI_markers['names'] = scVI_markers.index
scVI_markers

#### Let's start the identification

Let's plot the UMAP again with the legend on the clusters so it will be an easy reference.

In [ ]:
sc.pl.umap(integrated, color = ['leiden'], frameon = False, legend_loc = "on data")

Now we will create a dictionary from 0-18 since we have 19 clusters. As I go along, I will manually fill out the dictionary when I decide on what cell-type each cluster is. There are ways to automatically label clusters; however, it is found that doing it manually—alebit taking longer—is more accurate especially when you don't have a good reference dataset (which is definitely the case here since we are working with combined cyno and human lung data).

I just copy-pasted the output from above so we can fill in the cell-type for the key values as we identify clusters. Once we're done, we'll use this to map the cell labels of the AnnData.

In [ ]:
# create the dictionary
for x in range(0, 19):
    print(f"'{x}':'',")

In [ ]:
sc.pl.umap(integrated, color = ['leiden'], frameon = False, legend_loc = "on data")

It is easiest to start with cell-types that you know are going to be in the data. Almost every dataset contains some blood cells in them. These are a good indicator to reduce or increase the resolution of clustering. Let's look for those first: T cells, CD4+ T cells. We will look at expression as feature plots.

In [ ]:
sc.pl.umap(integrated, color = ['SCGB3A2', 'MARCO'], frameon = False, layer = 'scVI_normalized')

In [ ]:
sc.pl.umap(integrated, color = ['SCGB3A2', 'MARCO'], frameon = False, layer = 'scVI_normalized')

It seems like cluster 2 contains T cells. I switched out CD4 for CD8 to see if we needed to increase the resolution, but it seems to be fine as cluster 2 only contains CD4+ T cells.

Let's look for other cell types now

In [ ]:
sc.pl.umap(integrated, color = ['SCGB3A2', 'MARCO'], frameon = False, layer = 'scVI_normalized')

In [ ]:
scVI_markers[scVI_markers.names == 'SCGB3A2']

In [ ]:
scVI_markers[scVI_markers.group1 == '0'].sort_values(by = 'bayes_factor', ascending = False)

In [ ]:
{'0':'',
'1':'',
'2':'CD8+ T Cells',
'3':'',
'4':'',
'5':'',
'6':'',
'7':'',
'8':'AT2 Cells',
'9':'',
'10':'',
'11':'',
'12':'',
'13':'Ciliated Cells',
'14':'',
'15':'NK Cells',
'16':'',
'17':'AT1 Cells',
'18':'Dentritic Cells'}

#### Higher resolution needed

In [ ]:
sc.tl.leiden(integrated, resolution = 1)

Although we may have too many clusters, a lot of them will collapse.

In [ ]:
# integrated = sc.read_h5ad('integrated.h5ad')
# scVI_markers = pd.read_table('scVI_markers.tsv')
# scVI_markers = scVI_markers.set_index('names')
# scVI_markers = scVI_markers[(scVI_markers['is_de_fdr_0.05']) & (scVI_markers.lfc_mean) > 0.5]

In [ ]:
# scVI_markers = model.differential_expression(groupby = 'leiden')
# scVI_markers.to_csv('scVI_markers', sep = '\t', index = False)
# scVI_markers = scVI_markers[(scVI_markers['is_de_fdr_0.05']) & (scVI_markers.lfc_mean > .5)]
# # create a names column of the index so it is easier to filter the DF
# scVI_markers['names'] = scVI_markers.index
# scVI_markers

In [ ]:
sc.pl.umap(integrated, color = ['leiden'], frameon = False, legend_loc = "on data")

Get the markers

In [ ]:
sc.pl.umap(integrated, color = ['OLIG2'], frameon = False, layer = 'scVI_normalized', vmax = 5)

In [ ]:
scVI_markers[scVI_markers.index == 'PDGFRB'].sort_values(by = 'lfc_mean', ascending = False)

In [ ]:
scVI_markers[scVI_markers.group1 == 0].sort_values(by = 'lfc_mean', ascending = False)

In [ ]:
cell_type = {'0':'Monocytes',
'1':'Alveolar Macrophages',
'2':'CD4+ T Cells',
'3':'Alveolar Macrophages',
'4':'Neutrophils',
'5':'Alveolar Macrophages',
'6':'Monocytes',
'7':'CD8+ T Cells',
'8':'Alveolar Macrophages',
'9':'Monocytes',
'10':'Alveolar Macrophages',
'11':'Alveolar Macrophages',
'12':'Alveolar Macrophages',
'13':'Alveolar Macrophages',
'14':'AT2',
'15':'Granulocytes',
'16':'Monocytes',
'17':'Alveolar Macrophages',
'18':'Dendritic Cells',
'19':'Alveolar Macrophages',
'20':'Fibroblasts/Smooth Muscle Cells/Pericytes',
'21':'Monocytes',
'22':'B Cells',
'23':'Ciliated Cells',
'24':'Endothelial Cells',
'25':'NK/T Cells',
'26':'Alveolar Macrophages',
'27':'Club/Goblet/Mucous Cells',
'28':'Alveolar Macrophages',
'29':'Lymphatic Endothelial Cell',
'30':'AT1 Cells',
'31':'Neutrophils',
'32':'Endothelial Cells',
'33':'Mast Cells'}

#### Labelling now

In [ ]:
integrated.obs['cell_type'] = integrated.obs.leiden.map(cell_type)

Load the data and make new model

In [ ]:
sc.pl.umap(integrated, color = ['cell_type'], frameon = False)

Save markers to `.uns` slot in `integrated`

In [ ]:
integrated.uns['scVI_markers'] = scVI_markers
integrated.write_h5ad('integrated.h5ad')

In [ ]:
integrated = sc.read_h5ad('integrated.h5ad')
adata = integrated
del integrated

In [ ]:
adata = anndata.read_h5ad(path_to_anndata)
scvi.model.SCANVI.setup_anndata(adata, batch_key="batch", labels_key="labels")
vae = scvi.model.SCANVI(adata, "Unknown")
vae.train()
adata.obsm["X_scVI"] = vae.get_latent_representation()
adata.obs["pred_label"] = vae.predict()

## Cell Counts

Let's create some other plots as an exercise and also for some more metrics

Load the data

In [ ]:
adata = sc.read_h5ad('integrated.h5ad')

Making a dictionary of number of cells per sample

In [ ]:
total_cell_num = adata.obs.groupby('Sample').count()
total_cell_num = dict(zip(total_cell_num.index, total_cell_num.n_genes))
total_cell_num

In [ ]:
cell_type_counts = adata.obs.groupby(['Sample', 'cell_type']).count()
# get rid of rows that contain 0 and reset the index so they're not grouped together
cell_type_counts = cell_type_counts[cell_type_counts.sum(axis = 1) > 0].reset_index()
# all the columns are the same so we don't need all 16 of them
cell_type_counts = cell_type_counts[cell_type_counts.columns[0:3]]
cell_type_counts

Add total cells column using the dictionary we made above

In [ ]:
cell_type_counts['total_cells'] = cell_type_counts.Sample.map(total_cell_num).astype(int)
cell_type_counts = cell_type_counts.rename(columns = {'sample': 'number'})
cell_type_counts

Let's add a frequency column and species column

In [ ]:
def map_species(x):
    if '00' in x:
        return 'human'
    else:
        return 'cyno'

In [ ]:
cell_type_counts['frequency'] = cell_type_counts.number / cell_type_counts.total_cells
cell_type_counts['species'] = cell_type_counts.Sample.map(map_species)
# reordering the df
cell_type_counts = cell_type_counts[['Sample', 'species', 'cell_type', 'number', 'total_cells', 'frequency']]
cell_type_counts

#### Plots

In [ ]:
plt.figure(figsize = (11,4))

ax = sns.boxplot(data = cell_type_counts, x = 'cell_type', y = 'frequency', hue = 'species')

plt.xticks(rotation = 35, rotation_mode = 'anchor', ha = 'right')

plt.show()

## DE on AT1 and AT2 cells

#### Subset to wanted cells

In [ ]:
subset = adata[adata.obs['cell_type'].isin(['AT1 Cells', 'AT2'])].copy()

Let's see how many genes we have

In [ ]:
len(subset.var)

Going to refilter the subset. Some of these genes might be in a few number of cells or none at all.

In [ ]:
sc.pp.filter_genes(subset, min_cells = 100)

In [ ]:
len(subset.var)

That is a much better number

In [ ]:
model = scvi.model.SCVI.load('model/', adata, use_gpu = False)

In [ ]:
model

#### `SCVI` DE

In [ ]:
adata.obs['species'] = adata.obs.Sample.map(map_species)
adata.obs

Let's run this model to see how differentially expressed AT! and AT2 cells are between species

In [ ]:
# we need to pass a Boolean array for the two groups
de = model.differential_expression(
    idx1 = [(adata.obs['cell_type'].isin(['AT1 Cells', 'AT2'])) & (adata.obs.species == 'human')], 
    idx2 = [(adata.obs['cell_type'].isin(['AT1 Cells', 'AT2'])) & (adata.obs.species == 'cyno')]
    )

Let's look at the `SCVI` DE dataframe

In [ ]:
de.sort_values('lfc_mean', ascending = False)

**Based on this output, there's no differential expression really**

Let's try it by just cell type. This should be better

In [ ]:
scvi_de = model.differential_expression(
    idx1 = [adata.obs['cell_type'] == 'AT1 Cells'],
    idx2 = [adata.obs['cell_type'] == 'AT2']
    )

In [ ]:
scvi_de

Yep! Looks better :)

In [ ]:
scvi_de = scvi_de[(scvi_de['is_de_fdr_0.05']) & (abs(scvi_de.lfc_mean) > 1)]
scvi_de = scvi_de.sort_values('lfc_mean')
scvi_de

Some more filtering.

In [ ]:
scvi_de = scvi_de[(scvi_de.raw_normalized_mean1 > 0.5) | (scvi_de.raw_normalized_mean2 > 0.5)]
scvi_de

Saving...

In [ ]:
scvi_de.to_csv('scVI_diff_exp.tsv', sep = '\t', index = True)

Take the top and bottom 25 DE genes from sorted dataframe and plot heatmap

In [ ]:
DE_genes = scvi_de[-25:].index.tolist() + scvi_de[:25].index.tolist()

#### Heatmap of DE genes in AT1 and AT2
Plot as heatmap with `Scanpy`

In [ ]:
sc.pl.heatmap(subset, DE_genes, groupby = 'cell_type', swap_axes = True, layer = 'scVI_normalized', log = True)

#### GO Enrichment

In [ ]:
import gseapy as gp

Let's look at available gene sets

In [ ]:
gp.get_library_name()

In [ ]:
set_list = ['GO_Biological_Process_2023', 'GO_Cellular_Component_2023', 'GO_Molecular_Function_2023', 'KEGG_2021_Human']

Background genes will be from `subset` (all the `var.names`).

In [ ]:
enrich = gp.enrichr(gene_list = scvi_de[scvi_de.lfc_mean > 1].index.tolist(),
                   gene_sets = set_list, 
                   organism = 'human',
                   outdir = None,
                   background = subset.var_names.tolist())

In [ ]:
enrich.results

## Comparisons

#### Plots and stats

Unfortunately, ALPP and ALPG did not map to each other way back in the beginning. I should have checked for this and manually annotated them. Let's just use ALPL here which is in both data sets.

In [ ]:
sc.pl.violin(subset[subset.obs.cell_type == 'AT1 Cells'], 'ALPL', groupby = 'species')

In [ ]:
sc.pl.violin(subset[subset.obs.cell_type == 'AT2'], 'ALPL', groupby = 'species')

Let's do some stats

In [ ]:
from scipy import stats

This here is just finding the gene index (number) and then slicing twice. The reason we slice twice is bc the output is like `(array([167]),)` so the `[0][0]` just gets us the number

In [ ]:
# just setting this to a variable so I don't have to type it out every time
temp = subset[subset.obs.cell_type == 'AT1 Cells']

i = np.where(temp.var_names == 'ALPL')[0][0]

In [ ]:
a = temp[temp.obs.species == 'human'].X[:, i].toarray()
b = temp[temp.obs.species == 'cyno'].X[:, i].toarray()

Let's do a non-parametric test for AT1

In [ ]:
stats.mannwhitneyu(a, b)

We can see from the p-value that it isn't significant

Let's try the same for AT2

In [ ]:
temp = subset[subset.obs.cell_type == 'AT2']

i = np.where(temp.var_names == 'ALPL')[0][0]
a = temp[temp.obs.species == 'human'].X[:, i].toarray()
b = temp[temp.obs.species == 'cyno'].X[:, i].toarray()

In [ ]:
stats.mannwhitneyu(a, b)

Stll not significan't between the both of them!

## Score Gene Signature

#### `Scanpy`'s `score_genes` function.

This is super useful when elucidating cell types. You can pass a list of cell markers and see how well they score. The gene list should be above 10 genes for it to work.

In [ ]:
sc.tl.score_genes(subset, DE_genes, score_name = 'de')

In [ ]:
subset.obs.head()

All this did was add a column to our `.obs` data frame. This new column values are the relative expression of those genes from the list compared to the background. The numbers are arbitrary (so you can't compare between gene lists) but you can compare between cells (which is what we'll do next).

#### Violin Plots

Comparing between cell types

In [ ]:
sc.pl.violin(subset, 'de', groupby = 'cell_type')

Comparing between species

In [ ]:
sc.pl.violin(subset, 'de', groupby = 'species')

Comparing between samples

In [ ]:
sc.pl.violin(subset, 'de', groupby = 'Sample')

Can also run some stas with this `de` column

In [ ]:
a = subset[subset.obs.species == 'human'].obs.de.values
b = subset[subset.obs.species == 'cyno'].obs.de.values

stats.mannwhitneyu(a, b)

#### UMAP

You can also show this data via UMAP

In [ ]:
sc.pl.umap(subset, color = 'de', vmax = 0.5)

# The End

That's it! That is my attempt as a cross-species analysis. Now that you all have the basics, hopefully you can improve upon it. For example the orthologous genes could have maybe been done better, and also you can use `scANVI` (extension of `scVI` which there is some documentation for online) to do the cell type annotation. I decided to manually do it since we are working with two species, but I guess it may not matter too much. The next step is to take this AnnData and hopefully build a prototype Shiny app in R.

I want to say a big thanks to Kari for this opportunity this summer (2023). I really appreciate it! I learned a lot on this project and it was quite fun! I hope that you guys in iTox and Computational Toxicology use this as a resource in the future!


-Dan Carvalho

### Saving the AnnData, as well as counts, genes, cells, and metadata for export to R

In [ ]:
adata.write_h5ad('integrated.h5ad')

In [ ]:
integrated_counts = adata.X
integrated_genes = adata.var_names.tolist()
integrated_cells = adata.obs_names.tolist()
observations = adata.obs
var = adata.var

#### Save counts (sparse matrix) to disc

In [ ]:
from scipy.sparse import save_npz
save_npz('integrated_counts.npz', integrated_counts)

#### Save gene names as CSV

In [ ]:
ig = pd.DataFrame({'gene': integrated_genes})

ig.to_csv('integrated_genes.csv', sep = ',', index = False)

#### Save cell names as CSV

In [ ]:
ig = pd.DataFrame({'cell': integrated_cells})

ig.to_csv('integrated_cells.csv', sep = ',', index = False)

#### Metadata

Will save as TSV

In [ ]:
observations.to_csv('integrated_obs.tsv', sep = '\t', index = True)
var.to_csv('integrated_var.tsv', sep = '\t', index = True)

In [3]:
adata = sc.read_h5ad('cs_analysis/data/integrated.h5ad')

In [12]:
adata.X = adata.X.todense()

In [13]:
adata.X

matrix([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         1.8483456],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]], dtype=float32)

In [ ]:
adata.write(filename = 'cs_analysis/data/integrated_dense.h5ad')